## 13.4 Binary search

In Chapter&nbsp;11 you saw that searching for an item *x* in an ascending sequence
is often faster than in an unsorted sequence because we can stop when
the current item is larger than *x*:
at that point we know *x* can't be in the rest of the sequence.

We can search in a sorted sequence much faster with **binary search**.
It's a generate-and-test algorithm that generates very few candidates because
it decreases the search space by half after testing the current candidate.
Binary search is a decrease-and-conquer algorithm and not an exhaustive search
because it doesn't go through all candidates.

### 13.4.1 Recursive, with slicing

Binary search implements the membership operation on sorted sequences.
If I'm looking for 4 in the ascending sequence (..., 5, ...),
then the number must be in the left part, before the 5, if it exists.
If the sequence is in descending order instead,
then the 4 must be in the right part, after the 5.
More generally, inspecting the middle element of a sorted sequence
will always discard half of the items no matter where the sought item is
and whether the order is ascending or descending.

#### Algorithm

The algorithm stops when it reaches one of two base cases:
either the middle element is the sought element or the sequence is empty.
Here's the recursive definition for has(*items*, *item*),
assuming that the sequence is in ascending order.

- if *items* is empty: has(*items*, *item*) = false
- if *item* = middle element of *items*: has(*items*, *item*) = true
- if *item* < middle element of *items*: has(*items*, *item*) = has(left half of *items*, *item*)
- if *item* > middle element of *items*: has(*items*, *item*) = has(right half of *items*, *item*)

I defined the function in informal terms to convey the gist of binary search.
The algorithm computes the middle element and slices each half as done before
for [computing the maximum](../12_Recursion/12_7_multiple.ipynb#12.7.1-Dividing-the-input).

1. let *n* be │*items*│
1. if *n* = 0:
   1. let *exists* be false
1. otherwise:
   1. let *middle* be floor(*n* / 2)
   2. let *middle item* be *items*[*middle*]
   3. if *middle item* = *item*:
      1. let *exists* be true
   4. otherwise if *item* < *middle item*:
      1. let *exists* be has(*items*[0:*middle*], *item*)
   5. otherwise:
      1. let *exists* be has(*items*[*middle* + 1:*n*], *item*)

Contrary to the [multiple recursion membership algorithm](../12_Recursion/12_7_multiple.ipynb#Exercise-12.7.1),
this one searches only one of the halves: it's a single-recursion algorithm.
The difference between multiple and single recursion is not about
how many recursive calls are in the algorithm, but how many are executed.

Steps 2 and 3.3 check for the base cases; steps 3.4.1 and 3.5.1
decrease the sequence and recur. There's no combination step,
so the algorithm is tail recursive: the last step is either 3.4.1 or 3.5.1.

Neither half includes the middle item
(remember that the last index of a slice isn't included), so
each recursive call does indeed reduce the size of the input.
We don't need another base case.

#### Exercise 13.4.1

How would you change the algorithm if the sequence were in descending order?
Just describe the change briefly: don't write the full algorithm.

_Write your answer here._

[Answer](../32_Answers/Answers_13_4_01.ipynb)

#### Complexity

In the best-case scenario, the middle element is the sought one:
the algorithm takes constant time.

In a worst-case scenario, the sequence doesn't contain the item.
We can define the complexity T(*n*) recursively, using *n* = │*items*│.
The base case (steps 2 and 2.1) takes constant time, so T(0) = Θ(1).
For a non-empty sequence, each call takes

- Θ(1) for steps 3.1 to 3.3 (which fails in the worst case) and 3.4 or 3.5
- Θ(*n* / 2) for slicing half the sequence in either 3.4.1 or 3.5.1
- T(*n* / 2) for the recursive call in the same step.

Putting it all together:

- if *n* = 0: T(*n*) = Θ(1)
- if *n* > 0: T(*n*) = Θ(1) + Θ(*n* / 2) + T(*n* / 2) = T(*n* / 2) + Θ(*n*).

It has been proven that this leads to T(*n*) = Θ(*n*).

<div class="alert alert-warning">
<strong>Note:</strong> If T(0) = Θ(1) and T(<em>n</em>) = T(<em>n</em> / 2) + Θ(<em>n</em>), then T(<em>n</em>) = Θ(<em>n</em>).
</div>

### 13.4.2 Recursive, without slicing

Binary search with slicing is no better than linear search
in terms of complexity and is much worse in terms of memory due to all
the intermediate slices created.
To make the algorithm more efficient, I keep track of the start and end indices
of the slice to [avoid creating it](../12_Recursion/12_6_avoid_slicing.ipynb#12.6-Avoiding-slicing).

Here's the recursive binary search algorithm for membership function
has(*items*, *item*, *start*, *end*), with 0 ≤ *start* ≤ *end* ≤ │*items*│.

1. if *start* = *end*:
   1. let *exists* be false
2. otherwise:
   1. let *middle* be *start* + floor((*end* – *start*) / 2)
   1. let *middle item* be *items*[*middle*]
   1. if *middle item* = *item*:
      1. let *exists* be true
   2. otherwise if *item* < *middle item*:
      1. let *exists* be has(*items*, *item*, *start*, *middle*)
   2. otherwise:
      1. let *exists* be has(*items*, *item*, *middle* + 1, *end*)

I code the algorithm with an inner function to hide the
extra arguments from the user.
I add a print statement that shows the sequence searched in each call.
Feel free to uncomment it and to also print the middle item.

In [1]:
def has(items: list, item: object) -> bool:
    """Return True if and only if item is a member of items.

    Preconditions:
    - items is in ascending order
    - item is comparable to all members of items
    """

    def in_slice(start: int, end: int) -> bool:
        """Return True if and only if item is in slice items[start:end].

        Preconditions: 0 <= start <= end <= len(items)
        """
        # print('Searching', item, 'in', items[start:end])
        if end == start:
            return False
        else:
            middle = start + (end - start) // 2
            middle_item = items[middle]
            if middle_item == item:
                return True
            elif item < middle_item:
                return in_slice(start, middle)
            else:
                return in_slice(middle + 1, end)

    return in_slice(0, len(items))

Here are some tests. You may wish to add more, e.g. with all items the same.

In [2]:
from algoesup import test

has_tests = [
    # case,                 items,  item,   has?
    ('empty list',          [],     1,      False),
    ('is before 1 item',    [2],    1,      False),
    ('is the 1 item',       [1],    1,      True),
    ('is after 1 item',     [2],    3,      False),
    ('is before 2 items',   [2, 4], 1,      False),
    ('is between 2 items',  [2, 4], 3,      False),
    ('is after 2 items',    [2, 4], 5,      False),
    ('is 1st of 2 items',   [2, 4], 2,      True),
    ('is 2nd of 2 items',   [2, 4], 4,      True),
]

test(has, has_tests)

Testing has...
Tests finished: 9 passed (100%), 0 failed.


#### Exercise 13.4.2

Write the recursive definition of T for this algorithm's worst case and
determine the complexity by checking if the definition has a form seen before.

_Write your answer here._

[Hint](../31_Hints/Hints_13_4_02.ipynb)
[Answer](../32_Answers/Answers_13_4_02.ipynb)

You saw brute-force searches that don't generate symmetric candidates
in order to shrink the search space by half or better. For example, for the
[factorisation problem](../11_Search/11_2_factorisation.ipynb#11.2.3-Sort-candidates) we decreased
the upper limit of the search space from *n* to $\sqrt{n}$.
Binary search prunes the search space by half after *each* test,
not just at the start of the algorithm.
This makes a tremendous difference, as you've seen for the
[exponentiation operation](../13_Divide/13_2_decrease_half.ipynb#13.2.4-Code-and-performance).

### 13.4.3 Iterative

The recursive algorithm is tail recursive, so an iterative version is due.
It's not strictly necessary because a recursive decrease-by-half function
doesn't exceed the call stack limit: processing one billion items requires
at most $\log_2$ 10⁹ ≈ 30 calls.

An iterative implementation has the same complexity but is marginally faster,
as it avoids the constant-time function call overheads.
It simply reduces the slice until it's empty or the item is found.
I don't repeat the docstring.

In [3]:
def has_iterative(items: list, item: object) -> bool:  # noqa: D103
    start = 0
    end = len(items)
    while start < end:  # alternative: while start != end
        middle = start + (end - start) // 2
        middle_item = items[middle]
        if middle_item == item:
            return True
        elif item < middle_item:
            end = middle  # search left half [start:middle]
        else:
            start = middle + 1  # search right half [middle+1:end]
    return False


test(has_iterative, has_tests)

Testing has_iterative...
Tests finished: 9 passed (100%), 0 failed.


Let's compare the run-times for a worst case: the item isn't in the sequence.

In [4]:
# If running this cell in the VCE leads to a crash, set `items` to fewer zeros.
items = [0] * 1_000_000_000  # a billion zeros

%timeit -r 5 has(items, 1)
%timeit -r 5 has_iterative(items, 1)

3.08 μs ± 3.09 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)
2.27 μs ± 45.3 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)


Doing 30 iterations instead of 30 recursive calls only saves a few microseconds.
Recursion, even without tail optimisation, adds a very small overhead.
The efficiency of a search algorithm depends on how it prunes the search space,
not on whether it's recursive or iterative.

<div class="alert alert-warning">
<strong>Note:</strong> Recursion isn't inherently inefficient.
</div>

⟵ [Previous section](13_3_variable_decrease.ipynb) | [Up](13-introduction.ipynb) | [Next section](13_5_variants.ipynb) ⟶